# Form 13
This notebook describes how to use Neo4j and SageMaker together.  In it you connect to a Neo4j instance, load data and compute an embedding.  You then load that data into AWS S3.  Finally, you use SageMaker to train a model using the new embedding as an additional feature.

## Deploy Neo4j
You're going to need a Neo4j deployment to run this lab.  The easiest way to get that is via the [AWS Marketplace](https://aws.amazon.com/marketplace/seller-profile?id=23ec694a-d2af-4641-b4d3-b7201ab2f5f9).  Select "Neo4j Enterprise Edition" and deploy that.  Suggested parameters are:

* Graph Database Version - 4.4.8
* Install Graph Data Sceince - True
* Graph Data Science License Key - None
* Install Bloom - True
* Bloom License Key - None
* Password - Enter something here
* Node Count - 1
* Instance Type - t3.xlarge
* Key Name - Pick a key you have
* SSH CIDR - 0.0.0.0/0

The Marketplace listing deploys an ASG.  When deployment is complete, you can get the IP address of your Neo4j node from that ASG.  You can view deployed ASGs [here](https://us-east-1.console.aws.amazon.com/ec2autoscaling).

## Using the Neo4j API
Now that we have a Neo4j deployment, let's connect to Neo4j.  First off, install the Neo4j Graph Data Science package.

In [ ]:
%pip install graphdatascience

Now, you're going to need the connection string and credentials from the deployment you created above.

In [2]:
# Edit these variables!
DB_URL = 'neo4j://ec2-3-91-14-235.compute-1.amazonaws.com:7687'
DB_PASS = 'foo123'

# You can leave this default
DB_USER = 'neo4j'

In [3]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS))

## Load Data into Neo4j
Now that we've got our connection object, let's load the dataset into Neo4j.

The dataset is pulled from the SEC's EDGAR database. These are public filings of something called Form 13. Asset managers with over \$100m AUM are required to submit Form 13 quarterly. That's then made available to the public over http. The csvs linked above were pulled from EDGAR using some python scripts. If you're curious, they're all available [here](https://github.com/neo4j-partners/neo4j-sec-edgar-form13). We've filtered the data to only include filings over \$10m in value.

We're going to create constratints for our data.

In [ ]:
result = gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (p:Company) ASSERT (p.cusip) IS NODE KEY;')
display(result)

result = gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (p:Manager) ASSERT (p.filingManager) IS NODE KEY;')
display(result)

result = gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (p:Holding) ASSERT (p.filingManager, p.cusip, p.reportCalendarOrQuarter) IS NODE KEY;')
display(result)

Now let's load the nodes.

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MERGE (c:Company {cusip:row.cusip})
        ON CREATE SET
            c.nameOfIssuer=row.nameOfIssuer
    """
)
display(result)

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MERGE (m:Manager {filingManager:row.filingManager})
    """
)
display(result)

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MERGE (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
        ON CREATE SET
            h.value=row.value, 
            h.shares=row.shares,
            h.target=row.target,
            h.nameOfIssuer=row.nameOfIssuer
    """
)
display(result)

Now let's create relationships between those nodes.

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MATCH (m:Manager {filingManager:row.filingManager})
        MATCH (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
        MERGE (m)-[r:OWNS]->(h)
    """
)
display(result)

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MATCH (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
        MATCH (c:Company {cusip:row.cusip})
        MERGE (h)-[r:PARTOF]->(c)
    """
)
display(result)

## Graph Data Science
Now we're going to use Neo4j Graph Data Science to create an in memory graph represtation of the data.  We'll enhance that represation with features we engineer using a graph embedding.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.create(
      'mygraph',
      ['Company', 'Manager', 'Holding'],
      {
          OWNS: {orientation: 'UNDIRECTED'},
          PARTOF: {orientation: 'UNDIRECTED'}
      }
    )
    YIELD
      graphName AS graph,
      relationshipProjection AS readProjection,
      nodeCount AS nodes,
      relationshipCount AS rels
  """
)
display(result)

If you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.drop('mygraph')
  """
)
display(result)

Now, let's list the details of the graph to make sure the projection was created as we want.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.list()
  """
)
display(result)

Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

There are a bunch of parameters we could adjust in this.  One of the most obvious is the embeddingDimension.  The documentation covers many more.

In [ ]:
result = gds.run_cypher(
  """
  CALL gds.fastRP.mutate('mygraph',{
    embeddingDimension: 16,
    randomSeed: 1,
    mutateProperty:'embedding'
  })
  """
)
display(result)

That creates an embedding for each node type.  However, we only want the embedding on the nodes of type holding.

We're going to take the embedding from our projection and write it to the holding nodes in the underlying database.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.writeNodeProperties('mygraph', ['embedding'], ['Holding'])
    YIELD writeMillis
  """
)
display(result)

In [4]:
result = gds.run_cypher(
  """
    MATCH (n:Holding) RETURN n
  """
)
display(result)

,n
0,"(shares, cusip, reportCalendarOrQuarter, filin..."
1,"(shares, cusip, reportCalendarOrQuarter, filin..."
2,"(shares, cusip, reportCalendarOrQuarter, filin..."
3,"(shares, cusip, reportCalendarOrQuarter, filin..."
4,"(shares, cusip, reportCalendarOrQuarter, filin..."
...,...
446917,"(shares, cusip, reportCalendarOrQuarter, filin..."
446918,"(shares, cusip, reportCalendarOrQuarter, filin..."
446919,"(shares, cusip, reportCalendarOrQuarter, filin..."
446920,"(shares, cusip, reportCalendarOrQuarter, filin..."


Note that this query will take 2-3 minutes to run as it's grabbing nearly half a million nodes along with all their properties and our new embedding.

In [5]:
import pandas as pd
df = pd.DataFrame([dict(record.items()) for record in result['n']])
df

,shares,cusip,reportCalendarOrQuarter,filingManager,embedding,nameOfIssuer,value,target
0,270,88579Y101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[-0.06891357153654099, 0.17251430451869965, -0...",3M Co,52024000,False
1,195,00508Y102,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[-0.0012208909029141068, 0.005601376295089722,...",Acuity Brands Inc,32175000,False
2,4939,00724F101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[0.11818736791610718, 0.2625264525413513, -0.0...",Adobe Systems Inc,2347852000,False
3,1557,02079K305,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[0.14201897382736206, 0.26428067684173584, 0.1...",Alphabet Inc A,3211344000,False
4,837,02079K107,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,"[0.30085813999176025, 0.24750658869743347, 0.4...",Alphabet Inc C,1731443000,False
...,...,...,...,...,...,...,...,...
446917,56874,911312106,09-30-2021,LEE DANNER & BASS INC,"[-0.10756000876426697, 0.09883546084165573, 0....",United Parcel Svc. Cl B,10357000,False
446918,231000,92552v100,09-30-2021,LEE DANNER & BASS INC,"[-0.19772686064243317, 0.19481559097766876, 0....",ViaSat Inc,12721000,True
446919,55104,92826C839,09-30-2021,LEE DANNER & BASS INC,"[0.19159139692783356, 0.5284039974212646, 0.15...",Visa Inc,12274000,True
446920,79459,931142103,09-30-2021,LEE DANNER & BASS INC,"[0.16196903586387634, 0.5445767045021057, 0.60...",Wal-Mart Stores Inc.,11075000,False


Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [6]:
embeddings = pd.DataFrame(df['embedding'].values.tolist()).add_prefix("embedding_")
merged = df.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
merged

,shares,cusip,reportCalendarOrQuarter,filingManager,nameOfIssuer,value,target,embedding_0,embedding_1,embedding_2,...,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15
0,270,88579Y101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,3M Co,52024000,False,-0.068914,0.172514,-0.270718,...,0.142906,0.707541,0.255056,0.168236,0.014960,-0.183164,0.214030,-0.149799,0.436635,-0.393186
1,195,00508Y102,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Acuity Brands Inc,32175000,False,-0.001221,0.005601,-0.008561,...,-0.249999,-0.027272,0.210765,0.232817,0.275897,-0.276700,0.338566,-0.479243,0.789944,-0.336510
2,4939,00724F101,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Adobe Systems Inc,2347852000,False,0.118187,0.262526,-0.081295,...,-0.591563,0.398565,0.239687,-0.259427,-0.352596,0.025593,0.626399,0.247615,0.445566,0.094596
3,1557,02079K305,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Alphabet Inc A,3211344000,False,0.142019,0.264281,0.142079,...,-0.150988,1.035650,0.370033,-0.064532,-0.239375,-0.003771,0.124420,0.027277,0.077211,-0.477533
4,837,02079K107,03-31-2021,LEDERER & ASSOCIATES INVESTMENT COUNSEL/CA,Alphabet Inc C,1731443000,False,0.300858,0.247507,0.449530,...,-0.415947,0.365203,-0.428251,-0.000808,0.304490,0.002407,-0.100384,-0.317436,0.368293,-0.037012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446917,56874,911312106,09-30-2021,LEE DANNER & BASS INC,United Parcel Svc. Cl B,10357000,False,-0.107560,0.098835,0.136811,...,-0.246284,0.304195,0.127205,-0.927462,-0.280658,0.211018,-0.149063,-0.060732,-0.073806,-0.089820
446918,231000,92552v100,09-30-2021,LEE DANNER & BASS INC,ViaSat Inc,12721000,True,-0.197727,0.194816,0.100359,...,-0.246779,0.370053,0.573178,0.061018,-0.038240,-0.423521,-0.037658,-0.372200,-0.453317,-0.374626
446919,55104,92826C839,09-30-2021,LEE DANNER & BASS INC,Visa Inc,12274000,True,0.191591,0.528404,0.158589,...,-0.766427,-0.110385,0.576788,-0.022191,-0.768111,-0.192285,-0.119118,0.234878,0.124492,-0.344153
446920,79459,931142103,09-30-2021,LEE DANNER & BASS INC,Wal-Mart Stores Inc.,11075000,False,0.161969,0.544577,0.602678,...,-0.070697,-0.004854,0.217884,-0.519946,-0.045873,-0.047081,-0.103562,0.011971,-0.375597,-0.362491


Now that we have the data formatted properly, let's split it into training, testing and validation sets.  We'll write those to disk.

In [8]:
df = merged

train = df.loc[df['reportCalendarOrQuarter'] == '03-31-2021']
train.to_csv('train.csv', index=False)

test = df.loc[df['reportCalendarOrQuarter'] == '06-30-2021']
test.to_csv('test.csv', index=False)

validate = df.loc[df['reportCalendarOrQuarter'] == '09-30-2021']
validate.to_csv('validate.csv', index=False)

## SageMaker Connection
Let's setup our SageMaker connection.

In [15]:
import sagemaker
import boto3

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/form13'

role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Upload to Amazon S3
Now we're going to upload the training and testing data to our default SageMaker bucket.

In [16]:
train_data_s3_path = session.upload_data(path='train.csv', key_prefix=prefix + '/train')
print('Training data uploaded to: ' + train_data_s3_path)

test_data_s3_path = session.upload_data(path='test.csv', key_prefix=prefix + '/test')
print('Testing data uploaded to: ' + test_data_s3_path)

Training data uploaded to: s3://sagemaker-us-east-1-159878781974/sagemaker/form13/train/train.csv
Testing data uploaded to: s3://sagemaker-us-east-1-159878781974/sagemaker/form13/test/test.csv


## Setting up the SageMaker Autopilot Job
After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset.

In [17]:
auto_ml_job_config = {'CompletionCriteria': {'MaxCandidates': 5}}

input_data_config = [
    {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://{}/{}/train'.format(bucket, prefix),
            }
        },
        'TargetAttributeName': 'target',
    }
]

output_data_config = {'S3OutputPath': 's3://{}/{}/output'.format(bucket, prefix)}

## Launching the SageMaker Autopilot Job
You can now launch the Autopilot job by calling the create_auto_ml_job method.

In [18]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-form13-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(
    AutoMLJobName=auto_ml_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig=auto_ml_job_config,
    RoleArn=role,
)

AutoMLJobName: automl-form13-22-10-55-29


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:159878781974:automl-job/automl-form13-22-10-55-29',
 'ResponseMetadata': {'RequestId': '702afbb7-0d56-4d11-b31a-8b77f5afe841',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '702afbb7-0d56-4d11-b31a-8b77f5afe841',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Wed, 22 Jun 2022 10:55:29 GMT'},
  'RetryAttempts': 0}}

## Tracking SageMaker Autopilot job progress
SageMaker Autopilot job consists of the following high-level steps : * Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets. * Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level. * Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline).

In [ ]:
print('JobStatus - Secondary Status')
print('------------------------------')

describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(describe_response['AutoMLJobStatus'] + ' - ' + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']

    print(
        describe_response['AutoMLJobStatus'] + ' - ' + describe_response['AutoMLJobSecondaryStatus']
    )
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProg

## Results
Now use the describe_auto_ml_job API to look up the best candidate selected by the SageMaker Autopilot job.

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print('CandidateName: ' + best_candidate_name)
print('FinalAutoMLJobObjectiveMetricName: ' + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print('FinalAutoMLJobObjectiveMetricValue: ' + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

## Perform batch inference using the best candidate
Now that you have successfully completed the SageMaker Autopilot job on the dataset, create a model from any of the candidates by using Inference Pipelines.

In [ ]:
model_name = 'automl-form13-model-' + timestamp_suffix
model = sm.create_model(Containers=best_candidate['InferenceContainers'], ModelName=model_name, ExecutionRoleArn=role)
print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

You can use batch inference by using Amazon SageMaker batch transform. The same model can also be deployed to perform online inference using Amazon SageMaker hosting.



In [ ]:
transform_job_name = 'automl-form13-transform-' + timestamp_suffix

transform_input = {
    'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': test_data_s3_path}},
    'ContentType': 'text/csv',
    'CompressionType': 'None',
    'SplitType': 'Line',
}

transform_output = {
    'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket, prefix),
}

transform_resources = {'InstanceType': 'ml.m5.4xlarge', 'InstanceCount': 1}

sm.create_transform_job(
    TransformJobName=transform_job_name,
    ModelName=model_name,
    TransformInput=transform_input,
    TransformOutput=transform_output,
    TransformResources=transform_resources,
)

Watch the transform job for completion.

In [ ]:
print('JobStatus')
print('----------')

describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print(job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print(job_run_status)
    sleep(30)


Now let’s view the results of the transform job:

In [ ]:
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix)
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path)

data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)  # Keep the output on one page
data

## View other candidates explored by SageMaker Autopilot
You can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by SageMaker Autopilot and sort them by their final performance metric.

In [ ]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
    print(
        str(index)
        + '  '
        + candidate['CandidateName']
        + '  '
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value'])
    )
    index += 1


## Candidate Generation Notebook
Sagemaker AutoPilot also auto-generates a Candidate Definitions notebook. This notebook can be used to interactively step through the various steps taken by the Sagemaker Autopilot to arrive at the best candidate. This notebook can also be used to override various runtime parameters like parallelism, hardware used, algorithms explored, feature extraction scripts and more.

The notebook can be downloaded from the following Amazon S3 location:


In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']

## Data Exploration Notebook
Sagemaker Autopilot also auto-generates a Data Exploration notebook, which can be downloaded from the following Amazon S3 location:


In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']

## Cleanup
The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. This operation deletes all the generated models and the auto-generated notebooks as well.

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket)

job_outputs_prefix = '{}/output/{}'.format(prefix, auto_ml_job_name)
bucket.objects.filter(Prefix=job_outputs_prefix).delete()